In [1]:
import numpy as np
import matplotlib
from mido import Message, MetaMessage
import mido
import sys
import random
import os
import time
import math
pwd = "/home/zedroman/Documents/Sonia_Data/Songs/simpledata/"
savepwd="/home/zedroman/Documents/Sonia_Data/Songs/"
music = sorted([f for f in os.listdir(pwd) if f[len(f)-4:len(f)] == ".mid"])
sos = 0
eos = 1
message_type = {"note_on":1, "note_off":0}
type_message = {1:"note_on", 0:"note_off"}
tempo = 500000 * 5
hidden_size = 1000

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device="cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def create_random(music):
    randoms = []
    for i in range(len(music)):
        randoms.append([random.random() * 10 for i in range(hidden_size)])
    return randoms
random_starts = create_random(music)

In [4]:
# 0(sos) 1(eos) 2 - 129(note_on) 130 - 257(note_off) 258 - 385(velocity) 386-585 //386-395,396-405, 406-415, 416-425 (time_shift)

def encode_type(tp, num):
    if (tp == "sos"):
        return 0
    if (tp == "eos"):
        return eos
    if (tp == "note_on"):
        return num + 2
    if (tp == "note_off"):
        return num + 130
    if (tp == "velocity"):
        return num + 258
    if (tp == "time"):
        return (num // 8) + 386
    return -1

def get_type(c):
    if(c == 0):
        return "sos", c
    if(c == 1):
        return "eos", c
    if(c > 1 and c <= 129):
        return "note_on", c - 2
    
    if(c >= 130 and c < 258):
        return "note_off", c - 130
    
    if(c>=258 and c < 386):
        return "velocity", c - 258
    
    if(c>=386 and c < 586):
        return "time", int((c - 386) * 8)
    
    print(c)
    return "wtf", -1
    


def midi_to_list(mid=None, pwd=None, tensor=True):
    if(mid==None):
          mid = mido.MidiFile(pwd)
    sample=[]
    velocity = -1
    for i, track in enumerate(mid.tracks):
        for msg in track:
            if (msg.type == "note_off" or msg.type == "note_on"):
                if (msg.time != 0):
                    c= encode_type("time", msg.time)
                    if (c != -1):
                        sample.append(c)
#                         for i in c:
#                             sample.append(i)
                if (msg.velocity != velocity):
                    c = encode_type("velocity", msg.velocity)
                    if (c != -1): 
                        sample.append(c)
                    velocity = msg.velocity
                if (msg.type == "note_on"):
                    c = encode_type("note_on", msg.note)
                    if (c != -1): 
                        sample.append(c)
                if (msg.type == "note_off"):
                    c = encode_type("note_off", msg.note)
                    if (c != -1): 
                        sample.append(c)
    sample.append(eos)
    if (tensor):
        return torch.tensor(sample, device=device)
    else:
        return sample
#     return sorted(sample, key=lambda x: x[-1])

def check_midi(sample):
    note_on = {}
    dele = []
    for j,i in enumerate(sample):
#         print(i)
        tp, nt = get_type(i)
        if (tp == "note_on"):
            if(nt in note_on):
                dele.append(j);
            note_on[nt] = 1;
        if (tp == "note_off"):
            if(nt in note_on):
                del note_on[nt]
            else:
                dele.append(j)
    for i in range(len(dele) - 1, -1, -1):
        del(sample[dele[i]])
    for i in note_on:
        sample.append(encode_type("note_off", i))

def list_to_midi(sample):
    print("was: ",len(sample))
    check_midi(sample)
    print("now: ",len(sample))
    
    midi = mido.MidiFile(type=0)
    tr = mido.MidiTrack()
    midi.tracks.append(tr)
    tr.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    velocity = 0
    time = 0
    for i in sample:
        tp, nt = get_type(i)
        if (tp == "velocity"):
            velocity = nt
        if (tp == "time"):
            time += nt
        if (tp == "note_off" or tp == "note_on"):
#             print(nt, velocity, time)
            tr.append(Message(tp, note=nt, velocity=velocity, time=time, channel=0))
            time = 0
    return midi

In [5]:
for file in (music):
    midi_file = mido.MidiFile(pwd + file)
    sample = midi_to_list(mid=midi_file, tensor=False)
    print(sample)
    midi = list_to_midi(sample)

[398, 343, 54, 61, 81, 78, 73, 69, 66, 398, 322, 182, 189, 209, 206, 201, 197, 194, 398, 343, 66, 69, 73, 78, 81, 61, 54, 398, 322, 194, 197, 201, 206, 209, 189, 182, 398, 343, 54, 61, 81, 78, 73, 69, 66, 398, 322, 182, 189, 209, 206, 201, 197, 194, 398, 343, 66, 69, 73, 78, 81, 61, 54, 398, 322, 194, 197, 201, 206, 209, 189, 182, 398, 343, 50, 57, 69, 65, 62, 398, 322, 178, 185, 197, 193, 190, 398, 343, 62, 65, 69, 57, 50, 398, 322, 190, 193, 197, 185, 178, 398, 343, 50, 57, 69, 65, 62, 398, 322, 178, 185, 197, 193, 190, 398, 343, 62, 65, 69, 57, 50, 398, 322, 190, 193, 197, 185, 178, 1]
was:  129
now:  129
[385, 61, 80, 68, 65, 390, 322, 208, 387, 366, 73, 392, 322, 201, 352, 77, 392, 322, 205, 369, 75, 392, 322, 193, 385, 65, 389, 322, 203, 349, 85, 389, 322, 213, 385, 77, 392, 322, 205, 349, 77, 389, 322, 205, 364, 78, 389, 322, 206, 385, 77, 389, 322, 205, 389, 189, 196, 193, 385, 60, 63, 56, 392, 72, 395, 322, 200, 389, 362, 73, 392, 322, 188, 385, 61, 389, 322, 201, 389, 373, 75

[358, 45, 57, 389, 322, 173, 185, 395, 358, 76, 72, 69, 389, 322, 204, 200, 197, 401, 358, 57, 45, 389, 322, 185, 173, 389, 358, 76, 72, 69, 389, 322, 204, 200, 197, 395, 358, 57, 45, 389, 322, 185, 173, 395, 358, 69, 72, 76, 389, 322, 197, 200, 204, 394, 358, 45, 57, 389, 322, 173, 185, 388, 358, 57, 45, 389, 322, 185, 173, 389, 358, 69, 72, 76, 389, 322, 197, 200, 204, 394, 358, 57, 45, 389, 322, 185, 173, 395, 358, 69, 72, 76, 389, 322, 197, 200, 204, 400, 358, 45, 57, 389, 322, 173, 185, 389, 358, 76, 72, 69, 389, 322, 204, 200, 197, 394, 358, 57, 45, 76, 389, 322, 185, 173, 392, 204, 389, 358, 74, 72, 69, 389, 322, 200, 197, 387, 202, 387, 358, 45, 57, 386, 74, 388, 322, 173, 185, 387, 202, 387, 358, 72, 392, 57, 45, 389, 322, 200, 185, 173, 389, 358, 69, 69, 72, 389, 322, 197, 200, 388, 358, 57, 45, 389, 322, 185, 173, 389, 197, 1]
was:  180
now:  178
[358, 85, 76, 73, 69, 389, 322, 213, 358, 86, 389, 322, 214, 358, 85, 389, 322, 213, 358, 83, 389, 322, 211, 358, 81, 389, 322, 20

In [6]:

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [7]:
max_len = 1000
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, num):
        if (num != -1):
            random.seed(num)
        rand = random.random() * 10
#         if (num == -1):
        print(rand)
        return  torch.tensor([rand]*decoder.hidden_size, device=device).view(1,1,-1)

In [8]:
teacher_forcing_ratio = 1


def train(target_tensor, decoder, decoder_optimizer, criterion, max_length=max_len, num = -1):
    decoder_optimizer.zero_grad()
    target_length = target_tensor.size(0)

    loss = 0

    decoder_input = torch.tensor([[sos]], device=device)
    decoder_hidden =   torch.tensor(random_starts[num], device=device).view(1,1,-1)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
#             if di == max_len:
#                 break
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di:di+1])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di:di+1])
            if decoder_input.item() == eos:
                break

    loss.backward()

    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
def trainIters(decoder, n_iters,decoder_optimizer, print_every=len(music), epoch=0):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = music[iter % (len(music))]
#         print(training_pair)
        midi_file = mido.MidiFile(pwd + training_pair)
        target_tensor = midi_to_list(mid=midi_file)
        loss = train(target_tensor,
                     decoder, decoder_optimizer, criterion, num = iter % (len(music)))
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [123]:
learning_rate = 0.000001
# decoder = DecoderRNN(hidden_size, 586).to(device)
decoder_optimizer = optim.Adam(decoder.parameters() , lr=learning_rate)


In [124]:
trainIters(decoder, 10000, decoder_optimizer)

0m 40s (- 111m 30s) (60 0%) 0.0083
1m 20s (- 110m 49s) (120 1%) 0.0082
2m 1s (- 110m 10s) (180 1%) 0.0082
2m 41s (- 109m 29s) (240 2%) 0.0081
3m 21s (- 108m 49s) (300 3%) 0.0081
4m 2s (- 108m 9s) (360 3%) 0.0081
4m 42s (- 107m 29s) (420 4%) 0.0081
5m 23s (- 106m 49s) (480 4%) 0.0080
6m 3s (- 106m 8s) (540 5%) 0.0080
6m 43s (- 105m 28s) (600 6%) 0.0080
7m 24s (- 104m 48s) (660 6%) 0.0080
8m 4s (- 104m 7s) (720 7%) 0.0080
8m 45s (- 103m 27s) (780 7%) 0.0079
9m 25s (- 102m 46s) (840 8%) 0.0079
10m 5s (- 102m 6s) (900 9%) 0.0079
10m 46s (- 101m 25s) (960 9%) 0.0079
11m 26s (- 100m 45s) (1020 10%) 0.0079
12m 7s (- 100m 5s) (1080 10%) 0.0079
12m 47s (- 99m 24s) (1140 11%) 0.0078
13m 27s (- 98m 44s) (1200 12%) 0.0078
14m 8s (- 98m 4s) (1260 12%) 0.0078
14m 48s (- 97m 23s) (1320 13%) 0.0078
15m 29s (- 96m 43s) (1380 13%) 0.0078
16m 9s (- 96m 2s) (1440 14%) 0.0078
16m 49s (- 95m 22s) (1500 15%) 0.0077
17m 30s (- 94m 42s) (1560 15%) 0.0077
18m 10s (- 94m 1s) (1620 16%) 0.0077
18m 51s (- 93m 21s)

KeyboardInterrupt: 

In [134]:
def evaluate(decoder,num=-1, max_length=max_len):
    with torch.no_grad():

        decoder_input = torch.tensor([[sos]], device=device)  # SOS
        decoder_hidden = [random.random() for i in range(hidden_size)]
        decoder_hidden =   torch.tensor(decoder_hidden, device=device).view(1,1,-1)

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            topv = topv.max().item() - topv.cpu()[0].numpy()
            summ = topv.sum()
            chosen = np.random.choice(topi[0].cpu(), p= topv / summ)
            decoder_input = torch.tensor(chosen, device=device)
            if decoder_input == eos:
#                 decoded_words.append('<EOS>')
#                 break
                decoder_input = torch.tensor([[sos]], device=device)
            else:
                decoded_words.append(chosen)

#             decoder_input = topi.data.topk(1)[1].squeeze().detach()

        return decoded_words

In [135]:
def generate(number, decoder):
    for i in range(number):
        new_music = evaluate(decoder)
        midi = list_to_midi(new_music)
        print(midi.length)
        midi.save(savepwd +"test" + str(i) + ".mid")
    return midi

In [136]:

last = generate(10, decoder)
for track in last.tracks:
    for msg in track:
        print(msg)

/home/zedroman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/home/zedroman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


was:  847
now:  847
42.5
was:  970
now:  970
28.791666666666632
was:  926
now:  926
23.25
was:  913
now:  912
34.999999999999986
was:  842
now:  838
33.45833333333333
was:  978
now:  981
28.749999999999964
was:  830
now:  827
22.291666666666668
was:  910
now:  909
24.5
was:  901
now:  903
24.374999999999993
was:  966
now:  960
28.4583333333333
<meta message set_tempo tempo=2500000 time=0>
note_on channel=0 note=62 velocity=100 time=0
note_on channel=0 note=59 velocity=100 time=0
note_on channel=0 note=54 velocity=100 time=0
note_on channel=0 note=47 velocity=100 time=0
note_off channel=0 note=62 velocity=64 time=168
note_off channel=0 note=59 velocity=64 time=0
note_off channel=0 note=54 velocity=64 time=0
note_off channel=0 note=47 velocity=64 time=0
note_on channel=0 note=71 velocity=100 time=0
note_on channel=0 note=68 velocity=100 time=0
note_on channel=0 note=64 velocity=100 time=0
note_on channel=0 note=59 velocity=100 time=0
note_on channel=0 note=56 velocity=100 time=0
note_on 

In [82]:

for i in new_music:
    print(get_type(i))

NameError: name 'new_music' is not defined

In [60]:
def test():
    midi = mido.MidiFile("/home/zedroman/Documents/Sonia_Data/Songs/test_from.mid")
#     midi = mido.MidiFile()
#     tr = mido.MidiTrack()
#     midi.tracks.append(tr)
#     tr.append(Message("note_on", velocity = 10, time= 1111, channel=0))
    sample = midi_to_list(midi, tensor=False)
#     for i in sample:
#         print(i)
    midi = list_to_midi(sample)
    midi.save("/home/zedroman/Documents/Sonia_Data/Songs/test_to.mid")
tempo = 500000 * 5
test()

was:  197
now:  197


In [ ]:
midi = mido.MidiFile(pwd + "2.mid")
for i, track in enumerate(midi.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
# tmp = music[0]
# music[0] =music[2]
# music[2] = tmp

In [ ]:
print(music)

In [ ]:
torch.save(decoder, '/home/zedroman/Documents/Sonia_Data/test_models4/decoder')

/home/zedroman/anaconda3/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [10]:
decoder = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/decoder')

RuntimeError: unexpected EOF, expected 8 more bytes. The file might be corrupted.